In [1]:
import sys
sys.path.append("../")
import io

from document_search.ocr import DocumentReader, EntityProcessor


In [2]:
documents = [
    "../data/SP-51.13330.2011.pdf",
    "../data/SP-60.13330.2020.pdf"
]


In [3]:
doc_reader = DocumentReader()

raw_documents = []
processed_docs = []
for document_path in documents:
    with open(document_path, 'rb') as f:
        raw_documents.append(io.BytesIO(f.read()))

    processed_doc, errors = doc_reader.read(document_path)
    print(document_path, errors)
    processed_doc.entities = EntityProcessor.merge_text_entities(processed_doc.entities)
    processed_doc.entities = EntityProcessor.filter_short_entities(processed_doc.entities)
    processed_docs.append(processed_doc)


../data/SP-51.13330.2011.pdf []
../data/SP-60.13330.2020.pdf []


In [4]:
from document_search.storages import DocumentStorageE5
from document_search.search import TextEntityEmbedderE5

embedder = TextEntityEmbedderE5()
storage = DocumentStorageE5(embedder)


/home/atom/igor/hacks/document-search/notebooks/../document_search/search/embedders.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  self.embedder = HuggingFaceEmbeddings(
/home/atom/anaconda3/envs/doc-search2/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
for doc, raw_doc in zip(processed_docs, raw_documents):
    storage.add_document(doc, raw_doc, pbar=True)


  0%|          | 0/54 [00:00<?, ?it/s]

55it [00:55,  1.00s/it]                        
120it [00:57,  2.08it/s]                         


In [6]:
from document_search.rag.retrievers import YandexGPTRetriever


In [7]:
rag_retriever = YandexGPTRetriever(embedder=embedder, storage=storage)


/home/atom/igor/hacks/document-search/notebooks/../document_search/rag/retrievers.py:26: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  self.llm_chain = LLMChain(llm=self.llm, prompt=self.prompt)


In [8]:
query = "какие трубопроводы не допускается прокладывать через помещения вентиляционного оборудования?"


In [17]:
answer = rag_retriever.retrieve_answer(query, k=1, context_length=0)
print(answer)


**Ответ на вопрос:**

Через помещения, в которых размещено вентиляционное оборудование, **не допускается прокладка трубопроводов с легковоспламеняющимися и горючими жидкостями и газами, а также канализационных трубопроводов, кроме ливневой канализации и водоотведения из вышележащих помещений**.

Это требование указано в пункте 10.6 документа.

**Обоснование:**

В пункте 10.3 указано, что помещения для вентиляционного оборудования должны быть расположены в пределах обслуживаемого пожарного отсека или за его пределами согласно сводам правил по пожарной безопасности.

Пункт 10.5 запрещает размещать под помещениями с массовым пребыванием людей помещения с пылеуловителями для сухой очистки взрывоопасных смесей.

В пункте 10.6 говорится, что через помещения для вентиляционного оборудования не допускаются трубопроводы с легковоспламеняющимися и горючими жидкостями и газами. Также запрещено прокладывать канализационные трубопроводы, кроме ливневой канализации и трубопроводов водоотведения из в

In [14]:
data, answer = rag_retriever.retrieve_answer_detailed(query, k=5, rag_k=1, context_length=0)


In [15]:
print(answer)


**Ответ:**

В соответствии с требованиями, изложенными в пункте 10.6, через помещения для вентиляционного оборудования **не допускается прокладывать следующие трубопроводы**:

а) с легковоспламеняющимися и горючими жидкостями и газами;

б) канализационные с прочистками и ревизиями (кроме трубопроводов ливневой канализации и водоотведения из вышележащих помещений для вентиляционного оборудования, в том числе от вентиляционного оборудования).

**Обоснование ответа**:

Данный пункт указывает на то, что через помещения, предназначенные для размещения вентиляционного оборудования, запрещено прокладывать трубопроводы, по которым транспортируются легковоспламеняющиеся и горючие жидкости и газы, а также канализационные трубопроводы с прочистками и ревизиями, за исключением ливневой канализации и трубопроводов, предназначенных для отвода сточных вод из вышележащих помещений, включая стоки от самого вентиляционного оборудования.

Информация о том, какие именно трубопроводы относятся к легковоспл